In [1]:
# <img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [72]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [3]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy        : 1.16.5


In [4]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# !pip freeze > requirements.txt

In [5]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [7]:
VERSION    = 16
DIR  = './Project_7_data/' # подключил к ноутбуку внешний датасет
VAL_SIZE   = 0.20   # 20%

marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

# Data

In [8]:
# !ls '../input'

In [9]:
train = pd.read_csv('./Preproject_7_data/all_auto_ru_09_09_2020.csv') # датасет для обучения модели 
test = pd.read_csv(DIR+'test.csv')
sample_submission = pd.read_csv(DIR+'sample_submission.csv')

train_1 = pd.read_csv(DIR+'auto_ru_2020_09_09.csv') # датасет для обучения модели
train_2 = pd.read_csv(DIR+'auto_ru_2020_10_19.csv') # датасет для обучения модели
train_3 = pd.read_csv(DIR+'auto_ru_2021_08_13.csv') # датасет для обучения модели
train_4 = pd.read_csv(DIR+'auto_ru_2021_08_29.csv') # датасет для обучения модели
train_5 = pd.read_csv(DIR+'auto_ru_2021_10_01.csv') # датасет для обучения модели

In [10]:
# train.head(5)

In [11]:
# train_1.info()

In [12]:
# test.head(5)

In [13]:
# test.info()

## Data Preprocessing

In [14]:
# ... 

In [15]:
train.dropna(subset=['productionDate','mileage'], inplace=True)
train.dropna(subset=['price'], inplace=True)

In [16]:
train_1.dropna(subset=['productionDate','mileage'], inplace=True)
train_1.dropna(subset=['price'], inplace=True)
train_2.dropna(subset=['productionDate','mileage'], inplace=True)
train_2.dropna(subset=['price'], inplace=True)
train_3.dropna(subset=['productionDate','mileage'], inplace=True)
train_3.dropna(subset=['price'], inplace=True)
train_4.dropna(subset=['productionDate','mileage'], inplace=True)
train_4.dropna(subset=['price'], inplace=True)
train_5.dropna(subset=['productionDate','mileage'], inplace=True)
train_5.dropna(subset=['price'], inplace=True)

In [17]:
# для baseline просто возьму пару схожих признаков без полной обработки
columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']
df_train = train[columns]
df_test = test[columns]

df_train_1 = train_1[columns]
df_train_2 = train_2[columns]
df_train_3 = train_3[columns]
df_train_4 = train_4[columns]
df_train_5 = train_5[columns]

In [42]:
y = train['price']

y_1 = train_1['price']
y_2 = train_2['price']
y_3 = train_3['price']
y_4 = train_4['price']
y_5 = train_5['price']

## Label Encoding

In [37]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

df_train_1['sample'] = 1 # помечаем где у нас трейн
df_train_2['sample'] = 1 # помечаем где у нас трейн
df_train_3['sample'] = 1 # помечаем где у нас трейн
df_train_4['sample'] = 1 # помечаем где у нас трейн
df_train_5['sample'] = 1 # помечаем где у нас трейн

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

data_1 = df_test.append(df_train_1, sort=False).reset_index(drop=True) # объединяем
data_2 = df_test.append(df_train_2, sort=False).reset_index(drop=True) # объединяем
data_3 = df_test.append(df_train_3, sort=False).reset_index(drop=True) # объединяем
data_4 = df_test.append(df_train_4, sort=False).reset_index(drop=True) # объединяем
data_5 = df_test.append(df_train_5, sort=False).reset_index(drop=True) # объединяем

In [38]:
for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data[colum] = data[colum].astype('category').cat.codes

for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data_1[colum] = data_1[colum].astype('category').cat.codes  
    data_2[colum] = data_2[colum].astype('category').cat.codes      
    data_3[colum] = data_3[colum].astype('category').cat.codes      
    data_4[colum] = data_4[colum].astype('category').cat.codes 
    data_5[colum] = data_5[colum].astype('category').cat.codes  

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123654 entries, 0 to 123653
Data columns (total 6 columns):
bodyType              123654 non-null int16
brand                 123654 non-null int8
productionDate        123654 non-null int64
engineDisplacement    123654 non-null int16
mileage               123654 non-null int64
sample                123654 non-null int64
dtypes: int16(2), int64(3), int8(1)
memory usage: 3.4 MB


In [77]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

X_1 = data_1.query('sample == 1').drop(['sample'], axis=1)
X_2 = data_2.query('sample == 1').drop(['sample'], axis=1)
X_3 = data_3.query('sample == 1').drop(['sample'], axis=1)
X_4 = data_4.query('sample == 1').drop(['sample'], axis=1)
X_5 = data_5.query('sample == 1').drop(['sample'], axis=1)

X_4.mileage = X_4.mileage.apply(lambda x: int(re.sub("\D", "", x)))

## Train Split

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X_5, y_5, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)


# # Model 1 : CatBoost
![] (https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [45]:
%%time
model = CatBoostRegressor(iterations = 5000,random_seed = RANDOM_SEED,eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],silent=True)
model.fit(X_train, np.log(y_train),eval_set=(X_test, np.log(y_test)),verbose_eval=0,
         use_best_model=True)

Wall time: 26.4 s


In [48]:
%%time
model_1 = CatBoostRegressor(iterations = 5000,random_seed = RANDOM_SEED,eval_metric='MAPE',custom_metric=['R2', 'MAE'],silent=True)
model_1.fit(X_train_1, np.log(y_train_1),eval_set=(X_test_1, np.log(y_test_1)),verbose_eval=0,use_best_model=True)

Wall time: 20.2 s


In [59]:
# model_2 = CatBoostRegressor(iterations = 5000,random_seed = RANDOM_SEED,eval_metric='MAPE',custom_metric=['R2', 'MAE'],silent=True)
# model_2.fit(X_train_2, np.log(y_train_2),eval_set=(X_test_2, np.log(y_test_2)),verbose_eval=0,use_best_model=True)

In [106]:
%%time
model_3 = CatBoostRegressor(iterations = 30000,random_seed = RANDOM_SEED,eval_metric='MAPE',custom_metric=['R2', 'MAE'],silent=True)
model_3.fit(X_train_3, np.log(y_train_3),eval_set=(X_test_3, np.log(y_test_3)),verbose_eval=0,use_best_model=True)

Wall time: 3min 2s


In [83]:
%%time
model_4 = CatBoostRegressor(iterations = 5000,random_seed = RANDOM_SEED,eval_metric='MAPE',custom_metric=['R2', 'MAE'],silent=True)
model_4.fit(X_train_4, np.log(y_train_4),eval_set=(X_test_4, np.log(y_test_4)),verbose_eval=0,use_best_model=True)

Wall time: 25.7 s


In [64]:
%%time
model_5 = CatBoostRegressor(iterations = 5000,random_seed = RANDOM_SEED,eval_metric='MAPE',custom_metric=['R2', 'MAE'],silent=True)
model_5.fit(X_train_5, np.log(y_train_5),eval_set=(X_test_5, np.log(y_test_5)),verbose_eval=0,use_best_model=True)

Wall time: 25.4 s


In [84]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [107]:
predict_test_1 = np.exp(model_1.predict(X_test_1))
predict_submission_1 = np.exp(model_1.predict(X_sub))
# predict_test_2 = np.exp(model_2.predict(X_test_2))
# predict_submission_2 = np.exp(model_2.predict(X_sub))
predict_test_3 = np.exp(model_3.predict(X_test_3))
predict_submission_3 = np.exp(model_3.predict(X_sub))
predict_test_4 = np.exp(model_4.predict(X_test_4))
predict_submission_4 = np.exp(model_4.predict(X_sub))
predict_test_5 = np.exp(model_5.predict(X_test_5))
predict_submission_5 = np.exp(model_5.predict(X_sub))

In [103]:
predict_test_3

array([ 884839.61682161, 6748678.64015837,  935393.30887686, ...,
        610367.31693531,  146307.77378115,  634479.38704248])

In [108]:
predict_test_3

array([ 909602.66834131, 7226455.8599095 ,  937950.25627104, ...,
        604669.54655204,  150641.29394215,  642241.51060373])

In [104]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 15.07%


In [110]:
print(f"Точность модели_1 по метрике MAPE: {(mape(y_test_1, predict_test_1))*100:0.2f}%")
print(f"Точность модели_3 по метрике MAPE: {(mape(y_test_3, predict_test_3))*100:0.2f}%")
print(f"Точность модели_4 по метрике MAPE: {(mape(y_test_4, predict_test_4))*100:0.2f}%")
print(f"Точность модели_5 по метрике MAPE: {(mape(y_test_5, predict_test_5))*100:0.2f}%")

Точность модели_1 по метрике MAPE: 14.63%
Точность модели_3 по метрике MAPE: 797567.64%
Точность модели_4 по метрике MAPE: 0.35%
Точность модели_5 по метрике MAPE: 6.29%


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [90]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
# sample_submission.head(10)

In [111]:
sample_submission['price'] = predict_submission_1
sample_submission.to_csv(f'submission_model_1.csv', index=False)
# sample_submission.head(10)
sample_submission['price'] = predict_submission_4
sample_submission.to_csv(f'submission_model_4.csv', index=False)
sample_submission['price'] = predict_submission_5
sample_submission.to_csv(f'submission_model_5.csv', index=False)

В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

# ===================== EX ==========================

In [23]:
train[train.brand.apply(lambda x: x in marks_for_parsing)].shape

(49309, 26)